## Build Functions for ELT

## Install Requirments (Updated on 10/2/2024, streamlit 1.39.0)
# !pip3 uninstall pandas pyarrow fireducks numpy -y
# !pip3 install -r requirements.txt

In [2]:
import pandas as pd
import numpy as np
from IPython.display import display
from yfinance import Ticker
from pykalman import KalmanFilter

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline

from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

from sklearn.model_selection import cross_val_predict

from datetime import datetime

import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Suppresses INFO and WARNING messages
import tensorflow as tf

from src import modules as f # f is for function
%load_ext autoreload
%autoreload 2

# Check if GPU is available
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

E0000 00:00:1733826811.208526   27407 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733826811.211676   27407 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Num GPUs Available: 1


### Download, Transform, and Modeling All in One

In [16]:
f.predictions('mu')

MU 5m Interval Timestamp: 2024-12-15 17:20:40 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,static,static,up
kelly_1:2.5,0.236923,0.186404,0.210092,0.114706
prob_up,0.228908,0.289535,0.284103,0.375
prob_static,0.532085,0.380696,0.43563,0.25
prob_down,0.239007,0.329768,0.280267,0.375
precision,0.454945,0.41886,0.43578,0.367647
recall,0.43949,0.40552,0.403397,0.369979
f1,0.447084,0.412082,0.418964,0.368809
support,"[473.0, 471.0, 473.0]","[473.0, 471.0, 473.0]","[473.0, 471.0, 473.0]","[473.0, 471.0, 473.0]"


MU 15m Interval Timestamp: 2024-12-15 17:20:43 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,static,static,up
kelly_1:2.5,0.209901,0.20101,0.2125,0.072611
prob_up,0.345139,0.355575,0.326218,0.625
prob_static,0.462394,0.387953,0.416151,0.375
prob_down,0.192467,0.256472,0.257631,0.0
precision,0.435644,0.429293,0.4375,0.33758
recall,0.55,0.53125,0.525,0.33125
f1,0.486188,0.47486,0.477273,0.334385
support,"[160.0, 160.0, 161.0]","[160.0, 160.0, 161.0]","[160.0, 160.0, 161.0]","[160.0, 160.0, 161.0]"


MU 1h Interval Timestamp: 2024-12-15 17:20:45 


...

,XGBoost,GradientBoosting,RandomForest,KNN
prediction,down,down,down,static
kelly_1:2.5,0.060472,0.069903,0.073728,0.066667
prob_up,0.23116,0.234249,0.294288,0.25
prob_static,0.370284,0.379212,0.341126,0.5
prob_down,0.398555,0.386539,0.364585,0.25
precision,0.328909,0.335645,0.338377,0.333333
recall,0.328424,0.356406,0.362297,0.418879
f1,0.328666,0.345714,0.349929,0.371242
support,"[676.0, 678.0, 679.0]","[676.0, 678.0, 679.0]","[676.0, 678.0, 679.0]","[676.0, 678.0, 679.0]"


MU 1wk Interval Timestamp: 2024-12-15 17:20:52 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,static,static,up
kelly_1:2.5,0.078261,0.032215,0.06962,0.113669
prob_up,0.249392,0.290247,0.272522,0.375
prob_static,0.428639,0.363528,0.401945,0.25
prob_down,0.321969,0.346225,0.325532,0.375
precision,0.341615,0.308725,0.335443,0.366906
recall,0.392857,0.328571,0.378571,0.369565
f1,0.365449,0.318339,0.355705,0.368231
support,"[138.0, 140.0, 139.0]","[138.0, 140.0, 139.0]","[138.0, 140.0, 139.0]","[138.0, 140.0, 139.0]"


MU 1mo Interval Timestamp: 2024-12-15 17:20:53 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,down,up,down
kelly_1:2.5,0.113333,0.011765,0.008333,0.109091
prob_up,0.294748,0.31285,0.345804,0.375
prob_static,0.507271,0.290466,0.330215,0.125
prob_down,0.197981,0.396684,0.32398,0.5
precision,0.366667,0.294118,0.291667,0.363636
recall,0.354839,0.344828,0.225806,0.275862
f1,0.360656,0.31746,0.254545,0.313725
support,"[31.0, 31.0, 29.0]","[31.0, 31.0, 29.0]","[31.0, 31.0, 29.0]","[31.0, 31.0, 29.0]"


In [ ]:
symbol='NVDA'
interval='1d'

# # Define Eastern Time Zone
# eastern = pytz.timezone('US/Eastern')

# # Get current time in Eastern Time Zone
# eastern_time = datetime.now(eastern)

# # Format the time to include hour, minute, and seconds
# time_stamp = eastern_time.strftime('%Y-%m-%d %H:%M:%S')

# print(f'DL Time: {time_stamp}')

# f.download(symbol, interval, period)
f.transform(symbol, interval)
curr_prediction, models, feature_names, classification_reports = f.model(symbol, interval)
predictions, prediction_probas = f.make_prediction(models, curr_prediction, feature_names)

f.predictions_summary(predictions, prediction_probas, classification_reports)

In [ ]:
models

In [ ]:
models['XGBoost'].feature_importances_

In [ ]:
feature_names

In [ ]:
merged_list = sorted(zip(feature_names, models['XGBoost'].feature_importances_),
                     key=lambda x: x[1], 
                     reverse=True
                    )
merged_list

In [ ]:
predictions

In [ ]:
classification_reports

### Hyperparameter Searches

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

def model(symbol, interval, search_type='none'):
    # Load data
    data = f.load_model_df(symbol, interval)
    data.dropna(inplace=True, axis=0)
    X = data.drop(columns=['direction'], axis=1)
    y = data['direction']
    
    # Print column names to check for issues
    print("Columns in X before preprocessing:")
    print(X.columns)
    
    # Remove duplicate columns
    X = X.loc[:, ~X.columns.duplicated()]
    
    # Check if categorical_features are present in X
    categorical_features = ['day_of_month', 'day_of_week', 'hour_of_day']
    missing_features = [col for col in categorical_features if col not in X.columns]
    if missing_features:
        print(f"Missing categorical features: {missing_features}")
    
    # Store current prediction data (last row)
    curr_prediction = X.iloc[-1].copy()

    # Drop last row from X and y to prevent the model from seeing it
    X = X.iloc[:-1]
    y = y.iloc[:-1]
    
    # Create the categorical transformer
    categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    
    # Create the preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, categorical_features)
        ],
        force_int_remainder_cols=False # This will include all other columns in the transformed output
    )
    
    # Define your models
    models = {
        'XGBoost': XGBClassifier(random_state=42, n_jobs=-1),
        'RandomForest': RandomForestClassifier(random_state=42, n_jobs=-1),
        'GradientBoosting': GradientBoostingClassifier(random_state=42, validation_fraction=0.25, n_iter_no_change=31),
        # 'LightGBM': LGBMClassifier(random_state=42,force_col_wise=True),
        'KNN': KNeighborsClassifier(n_neighbors=7, p=1,weights='distance')
    }
    
    # Hyperparameters to search
    param_grids = {
        'XGBoost': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [3, 5, 7, 9],
            'classifier__learning_rate': [0.01, 0.1, 0.2, 0.3]
        },
        'RandomForest': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [None, 10, 20, 30],
            'classifier__min_samples_split': [2, 5, 10, 13]
        },
        'GradientBoosting': {
            'classifier__n_estimators': [100, 200, 300, 400],
            'classifier__max_depth': [3, 5, 7, 13],
            'classifier__le||arning_rate': [0.01, 0.1, 0.2, 0.5]
        },
        'KNN': {
            'classifier__n_neighbors': [3, 5, 7, 13],
            'classifier__weights': ['uniform', 'distance'],
            'classifier__p': [1, 2]  # 1: Manhattan, 2: Euclidean
        }
    }
    
    # Create a function to get feature names after transformation
    def get_feature_names_out(column_transformer):
        feature_names = []
        for name, transformer, columns in column_transformer.transformers_:
            if transformer == 'drop' or transformer == 'passthrough':
                if transformer == 'passthrough':
                    feature_names.extend(columns)
                continue
            if hasattr(transformer, 'get_feature_names_out'):
                names = transformer.get_feature_names_out(columns)
                feature_names.extend(names)
            else:
                feature_names.extend(columns)
        return feature_names
    
    # Split data before preprocessing to avoid data leakage
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, stratify=y, test_size=0.2, random_state=42)
    
    # Fit the preprocessor on training data
    preprocessor.fit(X_train)
    
    # Transform training and test data
    X_train_transformed = preprocessor.transform(X_train)
    X_test_transformed = preprocessor.transform(X_test)
    
    # Get feature names after transformation
    feature_names = get_feature_names_out(preprocessor)
    
    # Convert transformed data to DataFrame
    X_train_transformed = pd.DataFrame(X_train_transformed, columns=feature_names)
    X_test_transformed = pd.DataFrame(X_test_transformed, columns=feature_names)
    
    # Transform curr_prediction
    curr_prediction_transformed = preprocessor.transform(
        curr_prediction.to_frame().T)
    curr_prediction_transformed = pd.DataFrame(
        curr_prediction_transformed, columns=feature_names)
    
    for model_name, model in models.items():
        # Create a pipeline with the classifier
        pipeline = Pipeline(steps=[
            ('classifier', model)
        ])
        
        # Get the parameter grid for the current model
        param_grid = param_grids.get(model_name, {})
        
        # Use GridSearchCV or RandomizedSearchCV
        if search_type == 'grid' and param_grid:
            search = GridSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
        elif search_type == 'random' and param_grid:
            search = RandomizedSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy',
                n_jobs=-1, n_iter=10, random_state=42)
        else:
            search = pipeline
        
        # Fit the model
        search.fit(X_train_transformed, y_train)
        
        # If using search, get the best estimator
        if search_type in ['grid', 'random'] and param_grid:
            best_model = search.best_estimator_
            print(f"Best parameters for {model_name}: {search.best_params_}")
            model = best_model.named_steps['classifier']
        else:
            model = search.named_steps['classifier']
        
        # Store the model
        models[model_name] = model
        
        # Predict on test data
        y_pred = search.predict(X_test_transformed)
        
        # Evaluate the model
        print(f"Model: {model_name}")
        print(classification_report(y_test, y_pred, zero_division=0))
    
    return curr_prediction_transformed, models, feature_names

In [ ]:
curr_prediction, models, feature_names = model('AMD', '5m', 'grid')